In [109]:
import os
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import sklearn
import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

In [110]:
#label_to_num dict
l2n_dict ={'no_relation': 0, 'org:top_members/employees': 1, 'org:members': 2, 'org:product': 3, 'per:title': 4, 'org:alternate_names': 5, 'per:employee_of': 6, 'org:place_of_headquarters': 7, 'per:product': 8, 'org:number_of_employees/members': 9, 'per:children': 10, 
         'per:place_of_residence': 11, 'per:alternate_names': 12, 'per:other_family': 13, 'per:colleagues': 14, 'per:origin': 15, 'per:siblings': 16, 'per:spouse': 17, 'org:founded': 18, 'org:political/religious_affiliation': 19, 'org:member_of': 20, 
         'per:parents': 21, 'org:dissolved': 22, 'per:schools_attended': 23, 'per:date_of_death': 24, 'per:date_of_birth': 25, 'per:place_of_birth': 26, 'per:place_of_death': 27, 'org:founded_by': 28, 'per:religion': 29}



#num_to_label dict
n2l_dict = {value:key for key, value in list(l2n_dict.items())}



#labels 숫자형태로 만들기(모범답안)
test_with_label = pd.read_csv("../data/test_with_label.csv")
labels = list(test_with_label["label"])

for i in range(len(labels)):
    labels[i] = l2n_dict[labels[i]]



# f1계산해주는 함수
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0



# 성능 계산해주는 함수
def score_calculator(file_list):
    scores = []
    for i in range(len(file_list)):
        
        temp = eval(f"df{i}")
        
        #preds를 숫자로 바꾸기
        preds = list(temp["pred_label"])
        for i in range(len(preds)):
            preds[i] = l2n_dict[preds[i]]
            
            
        scores.append(klue_re_micro_f1(preds, labels))
        
    return scores

# 앙상블 해주는 함수
def ensemble(file_list, num):
    for candidates in combinations(range(len(file_list)), num):
        temp = pd.DataFrame({"id":list(range(7765)) , "pred_label":range(7765), "probs":[[0.0]*30 for _ in range(7765)] })


        score_sum = 0
        for i in candidates:
            for j in range(7765):
                temp["probs"][j] = temp["probs"][j] + np.array(eval(eval(f"df{i}")["probs"][j]))*scores[i]

            score_sum += scores[i]


        for k in range(7765):

            #최종 temp의 pred_label 채워 놓기
            temp["pred_label"][k] = n2l_dict[np.argmax(temp["probs"][k])]

            #최종 temp의 probs 채워 놓기
            temp["probs"][k] = str(list(temp["probs"][k]/score_sum))



        #preds를 숫자로 바꾸기
        preds = list(temp["pred_label"])
        for i in range(len(preds)):
            preds[i] = l2n_dict[preds[i]] 


        print("들어간 모델: ", candidates)
        print(klue_re_micro_f1(preds, labels))
        print()
        
# 성능 계산해주는 함수
def submission(file_list, candidates):
    temp = pd.DataFrame({"id":list(range(7765)) , "pred_label":range(7765), "probs":[[0.0]*30 for _ in range(7765)] })
    score_sum = 0
    
    for i in candidates:
        for j in range(7765):
            temp["probs"][j] = temp["probs"][j] + np.array(eval(eval(f"df{i}")["probs"][j]))*scores[i]

            score_sum += scores[i]
            
    for k in range(7765):

        #최종 temp의 pred_label 채워 놓기
        temp["pred_label"][k] = n2l_dict[np.argmax(temp["probs"][k])]

        #최종 temp의 probs 채워 놓기
        temp["probs"][k] = str(list(temp["probs"][k]/score_sum))
        
    return temp

In [112]:
# ensemble 폴더에 있는 csv 파일이름 담긴 리스트 만들기
# origin = "/Users/kowoojin/Desktop/앙상블 후보군/selected"
origin = os.getcwd()
# origin = "/Users/kowoojin/Desktop/앙상블 후보군"
file_list = os.listdir(origin + "/submission")
#file_list.remove( 'selected')
#file_list.remove( '.DS_Store')
file_list.sort()
# file_list = file_list[1:]
print(len(file_list))
file_list

17


['final_0212.csv',
 'final_0213.csv',
 'final_026.csv',
 'final_027.csv',
 'final_0613.csv',
 'final_067.csv',
 'final_0713.csv',
 'final_1713.csv',
 'final_2413.csv',
 'final_246.csv',
 'final_24613.csv',
 'final_2467.csv',
 'final_246713.csv',
 'final_24713.csv',
 'final_26713.csv',
 'final_46713.csv',
 'final_713.csv']

In [119]:
i = [2,5,6,7,8,9,12,13,14,15]
file_list0 = [file_list[j] for j in i]

In [120]:
ensemble(file_list0, 4)

들어간 모델:  (0, 1, 2, 3)
71.75499841320216

들어간 모델:  (0, 1, 2, 4)
71.70111287758347

들어간 모델:  (0, 1, 2, 5)
71.83076188961347

들어간 모델:  (0, 1, 2, 6)
71.61157678317254

들어간 모델:  (0, 1, 2, 7)
72.0709309689677

들어간 모델:  (0, 1, 2, 8)
71.89086294416244

들어간 모델:  (0, 1, 2, 9)
71.58600862482032

들어간 모델:  (0, 1, 3, 4)
71.63187855787476

들어간 모델:  (0, 1, 3, 5)
71.94426852438252

들어간 모델:  (0, 1, 3, 6)
71.58126375353662

들어간 모델:  (0, 1, 3, 7)
71.94968553459118

들어간 모델:  (0, 1, 3, 8)
71.68798357808306

들어간 모델:  (0, 1, 3, 9)
71.8054672600127

들어간 모델:  (0, 1, 4, 5)
71.54394299287411

들어간 모델:  (0, 1, 4, 6)
71.70938826859569

들어간 모델:  (0, 1, 4, 7)
72.14409859377469

들어간 모델:  (0, 1, 4, 8)
71.92399049881236

들어간 모델:  (0, 1, 4, 9)
71.91441801053809

들어간 모델:  (0, 1, 5, 6)
71.76656151419559

들어간 모델:  (0, 1, 5, 7)
72.2389003002054

들어간 모델:  (0, 1, 5, 8)
72.14365167646591

들어간 모델:  (0, 1, 5, 9)
71.90795781399808

들어간 모델:  (0, 1, 6, 7)
72.23355831109714

들어간 모델:  (0, 1, 6, 8)
72.11977935382191

들어간 모델:  (0, 1, 6, 

들어간 모델:  (4, 6, 7, 8)
72.51775848460932

들어간 모델:  (4, 6, 7, 9)
72.68082388631646

들어간 모델:  (4, 6, 8, 9)
72.60514952822645

들어간 모델:  (4, 7, 8, 9)
72.47956403269754

들어간 모델:  (5, 6, 7, 8)
72.43670886075948

들어간 모델:  (5, 6, 7, 9)
72.619996810716

들어간 모델:  (5, 6, 8, 9)
72.80307889672869

들어간 모델:  (5, 7, 8, 9)
72.63697532842038

들어간 모델:  (6, 7, 8, 9)
72.7563078888534



In [113]:
# 각각의 파일을 불러오기(df_{i} 형식)
for i in range(len(file_list)):
    globals()[f"df{i}"] = pd.read_csv(f"{origin}/submission/{file_list[i]}")

In [114]:
# #models: df들을 모아 놓은 리스트
# models =[] 
# for i in range(len(file_list)):
#     models.append(globals()[f"df{i}"])
    
    
#scores: 각 df의 리더보드 점수를 모아 놓은 리스트
scores = score_calculator(file_list)
scores

[71.0688836104513,
 71.33698201720094,
 71.26657083532982,
 71.65540009432479,
 71.19236883942766,
 71.46267706509629,
 71.79167316388586,
 71.49335418295544,
 71.6531681753216,
 71.6297135458812,
 72.22489959839358,
 71.99871258448664,
 72.47956403269754,
 72.26438962681846,
 72.13529318792477,
 72.53521126760563,
 71.03858471862131]

# 앙상블 돌리는 코드!!!

In [115]:
print(file_list)
print(scores)

['final_0212.csv', 'final_0213.csv', 'final_026.csv', 'final_027.csv', 'final_0613.csv', 'final_067.csv', 'final_0713.csv', 'final_1713.csv', 'final_2413.csv', 'final_246.csv', 'final_24613.csv', 'final_2467.csv', 'final_246713.csv', 'final_24713.csv', 'final_26713.csv', 'final_46713.csv', 'final_713.csv']
[71.0688836104513, 71.33698201720094, 71.26657083532982, 71.65540009432479, 71.19236883942766, 71.46267706509629, 71.79167316388586, 71.49335418295544, 71.6531681753216, 71.6297135458812, 72.22489959839358, 71.99871258448664, 72.47956403269754, 72.26438962681846, 72.13529318792477, 72.53521126760563, 71.03858471862131]


In [26]:
ensemble(file_list, num=1)

들어간 모델:  (0, 1)
67.47437092264678

들어간 모델:  (0, 2)
66.89774696707104

들어간 모델:  (0, 3)
69.99685830977064

들어간 모델:  (0, 4)
67.1662763466042

들어간 모델:  (0, 5)
67.4414902085655

들어간 모델:  (0, 6)
67.22213488445213

들어간 모델:  (0, 7)
67.29117876658862

들어간 모델:  (0, 8)
67.46802280782863

들어간 모델:  (0, 9)
66.50054508643515

들어간 모델:  (0, 10)
67.15625

들어간 모델:  (0, 11)
67.331360024926

들어간 모델:  (0, 12)
66.10143363650377

들어간 모델:  (0, 13)
66.10143363650377

들어간 모델:  (0, 14)
66.10143363650377

들어간 모델:  (1, 2)
67.06249999999999

들어간 모델:  (1, 3)
69.59228135698724

들어간 모델:  (1, 4)
65.98410757946212

들어간 모델:  (1, 5)
67.0888079784708

들어간 모델:  (1, 6)
66.6252716547656

들어간 모델:  (1, 7)
67.39297222648459

들어간 모델:  (1, 8)
66.70621385140146

들어간 모델:  (1, 9)
65.0091519219036

들어간 모델:  (1, 10)
66.84041085390159

들어간 모델:  (1, 11)
66.47337097512589

들어간 모델:  (1, 12)
64.33338276731426

들어간 모델:  (1, 13)
64.33338276731426

들어간 모델:  (1, 14)
64.33338276731426

들어간 모델:  (2, 3)
69.54050213169114

들어간 모델:  (2, 4)
65.8742571

In [116]:
ensemble(file_list, num=2)

들어간 모델:  (0, 1)
71.49164301482182

들어간 모델:  (0, 2)
71.2988047808765

들어간 모델:  (0, 3)
71.22552116234996

들어간 모델:  (0, 4)
71.55788470700334

들어간 모델:  (0, 5)
71.56021607880521

들어간 모델:  (0, 6)
71.6422610612502

들어간 모델:  (0, 7)
72.15810276679842

들어간 모델:  (0, 8)
71.64131812420787

들어간 모델:  (0, 9)
71.73320506653839

들어간 모델:  (0, 10)
72.11860353897657

들어간 모델:  (0, 11)
72.07667731629394

들어간 모델:  (0, 12)
72.14262973575295

들어간 모델:  (0, 13)
71.95566112430721

들어간 모델:  (0, 14)
71.6540404040404

들어간 모델:  (0, 15)
72.65462681796389

들어간 모델:  (0, 16)
72.1875

들어간 모델:  (1, 2)
71.45581691036236

들어간 모델:  (1, 3)
71.20665306762905

들어간 모델:  (1, 4)
71.38568494232895

들어간 모델:  (1, 5)
71.47603227337447

들어간 모델:  (1, 6)
71.63852407754847

들어간 모델:  (1, 7)
71.99874784786351

들어간 모델:  (1, 8)
71.7981072555205

들어간 모델:  (1, 9)
71.629392971246

들어간 모델:  (1, 10)
71.847739888977

들어간 모델:  (1, 11)
72.22045237336732

들어간 모델:  (1, 12)
72.04437400950872

들어간 모델:  (1, 13)
72.41216322672128

들어간 모델:  (1, 14)
71.9546742

In [117]:
ensemble(file_list, num=3)

들어간 모델:  (0, 1, 2)
71.70050761421321

들어간 모델:  (0, 1, 3)
71.5952080706179

들어간 모델:  (0, 1, 4)
71.79974651457542

들어간 모델:  (0, 1, 5)
71.99239182120779

들어간 모델:  (0, 1, 6)
71.7408397546784

들어간 모델:  (0, 1, 7)
72.1548874547458

들어간 모델:  (0, 1, 8)
71.53906866614048

들어간 모델:  (0, 1, 9)
71.7443154714581

들어간 모델:  (0, 1, 10)
72.06208425720621

들어간 모델:  (0, 1, 11)
72.02031423583558

들어간 모델:  (0, 1, 12)
72.15711118150142

들어간 모델:  (0, 1, 13)
72.09082308420058

들어간 모델:  (0, 1, 14)
72.00504413619167

들어간 모델:  (0, 1, 15)
72.38367476576147

들어간 모델:  (0, 1, 16)
72.08866687480486

들어간 모델:  (0, 2, 3)
71.619471842189

들어간 모델:  (0, 2, 4)
71.97198790386757

들어간 모델:  (0, 2, 5)
71.83569495303297

들어간 모델:  (0, 2, 6)
71.9606910762403

들어간 모델:  (0, 2, 7)
72.23368788696618

들어간 모델:  (0, 2, 8)
71.7557251908397

들어간 모델:  (0, 2, 9)
71.73982697853253

들어간 모델:  (0, 2, 10)
71.83098591549295

들어간 모델:  (0, 2, 11)
71.8219660582773

들어간 모델:  (0, 2, 12)
71.76132737715379

들어간 모델:  (0, 2, 13)
72.04968944099377

들어간 모델:  (

KeyboardInterrupt: 

In [65]:
# [2,4,6,7,13] -> ()
ensemble(file_list, num=4)

들어간 모델:  (0, 1, 2, 3)
71.99871258448664

들어간 모델:  (0, 1, 2, 4)
72.22489959839358

들어간 모델:  (0, 1, 3, 4)
72.26438962681846

들어간 모델:  (0, 2, 3, 4)
72.13529318792477

들어간 모델:  (1, 2, 3, 4)
72.53521126760563



In [66]:
ensemble(file_list, num=5) # [2,4,6,7,13]

들어간 모델:  (0, 1, 2, 3, 4)
72.47956403269754



In [35]:
ensemble(file_list, num=6)

들어간 모델:  (0, 1, 2, 3, 4, 5)
71.1864406779661

들어간 모델:  (0, 1, 2, 3, 4, 6)
71.1864406779661

들어간 모델:  (0, 1, 2, 3, 4, 7)
71.1864406779661

들어간 모델:  (0, 1, 2, 3, 5, 6)
70.48472244440889

들어간 모델:  (0, 1, 2, 3, 5, 7)
70.48472244440889

들어간 모델:  (0, 1, 2, 3, 6, 7)
70.48472244440889

들어간 모델:  (0, 1, 2, 4, 5, 6)
71.15354017501988

들어간 모델:  (0, 1, 2, 4, 5, 7)
71.15354017501988

들어간 모델:  (0, 1, 2, 4, 6, 7)
71.15354017501988

들어간 모델:  (0, 1, 2, 5, 6, 7)
70.91053048297702

들어간 모델:  (0, 1, 3, 4, 5, 6)
70.46366115835072

들어간 모델:  (0, 1, 3, 4, 5, 7)
70.46366115835072

들어간 모델:  (0, 1, 3, 4, 6, 7)
70.46366115835072

들어간 모델:  (0, 1, 3, 5, 6, 7)
69.48114727682886

들어간 모델:  (0, 1, 4, 5, 6, 7)
70.93060441317559

들어간 모델:  (0, 2, 3, 4, 5, 6)
69.84377465677765

들어간 모델:  (0, 2, 3, 4, 5, 7)
69.84377465677765

들어간 모델:  (0, 2, 3, 4, 6, 7)
69.84377465677765

들어간 모델:  (0, 2, 3, 5, 6, 7)
68.48285580084547

들어간 모델:  (0, 2, 4, 5, 6, 7)
69.81989036805012

들어간 모델:  (0, 3, 4, 5, 6, 7)
68.377144656068

들어간 모델:  (1, 2, 3,

KeyboardInterrupt: 

In [34]:
ensemble(file_list, num=7)

들어간 모델:  (0, 1, 2, 3, 4, 5, 6)
71.1864406779661

들어간 모델:  (0, 1, 2, 3, 4, 5, 7)
71.1864406779661

들어간 모델:  (0, 1, 2, 3, 4, 6, 7)
71.1864406779661

들어간 모델:  (0, 1, 2, 3, 5, 6, 7)
70.48472244440889

들어간 모델:  (0, 1, 2, 4, 5, 6, 7)
71.15354017501988

들어간 모델:  (0, 1, 3, 4, 5, 6, 7)
70.46366115835072

들어간 모델:  (0, 2, 3, 4, 5, 6, 7)
69.84377465677765

들어간 모델:  (1, 2, 3, 4, 5, 6, 7)
71.06063029915214



In [33]:
ensemble(file_list, num=8)

들어간 모델:  (0, 1, 2, 3, 4, 5, 6, 7)
71.1864406779661



# to_csv

In [107]:
result = submission(file_list, candidates = [2,4,13])

In [108]:
result.to_csv("./final_2413.csv")